In [3]:
import numpy as np
import pandas as pd
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras import models

from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2

import sys
from PIL import Image
sys.modules['Image'] = Image 

from PIL import Image
print(Image.__file__)

import os
import shutil

Using TensorFlow backend.


/home/serg/anaconda3/envs/py36/lib/python3.6/site-packages/PIL/Image.py


In [4]:
data_dir = 'data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
validation_dir = os.path.join(data_dir, 'validation')

df_train = pd.read_csv('data/labels.csv')
df_test = pd.read_csv('data/sample_submission.csv')

im_size = 200

x_train = []
y_train = []
x_test = []


In [8]:

print(df_train.head(10))

targets_series = pd.Series(df_train['breed'])
# print(targets_series)
one_hot = pd.get_dummies(targets_series, sparse=True)

one_hot_labels = np.asarray(one_hot)


                                 id               breed
0  000bec180eb18c7604dcecc8fe0dba07         boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97               dingo
2  001cdf01b096e06d78e9e5112d419397            pekinese
3  00214f311d5d2247d5dfe4fe24b2303d            bluetick
4  0021f9ceb3235effd7fcde7f7538ed62    golden_retriever
5  002211c81b498ef88e1b40b9abf84e1d  bedlington_terrier
6  00290d3e1fdd27226ba27a8ce248ce85  bedlington_terrier
7  002a283a315af96eaea0e28e7163b21b              borzoi
8  003df8b8a8b05244b1d920bb6cf451f9             basenji
9  0042188c895a2f14ef64a918ed9c7b64  scottish_deerhound


In [6]:
# Create the base pre-trained model
# Can't download weights in the kernel
# base_model = VGG16(
#     weights='imagenet',
#     # weights = None,
#     include_top=False, input_shape=(im_size, im_size, 3))

base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(im_size, im_size, 3), pooling=None)
for layer in base_model.layers[:-4]:
    layer.trainable = True
# Create the model

model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(base_model)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(120, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()


/home/serg/anaconda3/envs/py36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               123000    
Total params: 126,472,184
Trainable params: 126,419,064
Non-trainable params: 53,120
_________________________________________________________________


In [7]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 15
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(im_size, im_size),
        batch_size=train_batchsize,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(im_size, im_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


Found 7422 images belonging to 120 classes.
Found 2800 images belonging to 120 classes.


In [8]:
# model.fit(X_train, Y_train, epochs=5, validation_data=(X_valid, Y_valid), verbose=1)
# Train the model


model.compile(loss='categorical_crossentropy', 
              # optimizer='adam', 
              optimizer=optimizers.RMSprop(lr=1e-4), 
              metrics=['accuracy'])


history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize ,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)
  
# Save the model
model.save('small_last4.h5')

Epoch 1/5


  1/494 [..............................] - ETA: 2:06:58 - loss: 8.0388 - acc: 0.0000e+00

  2/494 [..............................] - ETA: 1:14:59 - loss: 11.2824 - acc: 0.0000e+00

  3/494 [..............................] - ETA: 58:07 - loss: 12.8725 - acc: 0.0000e+00  

  4/494 [..............................] - ETA: 49:52 - loss: 13.5548 - acc: 0.0000e+00

...

490/494 [============================>.] - ETA: 14s - loss: 3.1426 - acc: 0.3992

491/494 [============================>.] - ETA: 10s - loss: 3.1419 - acc: 0.3994

492/494 [============================>.] - ETA: 7s - loss: 3.1418 - acc: 0.3994 

493/494 [============================>.] - ETA: 3s - loss: 3.1410 - acc: 0.3995

494/494 [==============================] - 1948s 4s/step - loss: 3.1402 - acc: 0.3995 - val_loss: 9.6036 - val_acc: 0.1961


In [ ]:

history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)


Epoch 1/3


  1/494 [..............................] - ETA: 45:07 - loss: 0.9508 - acc: 0.6667

  2/494 [..............................] - ETA: 37:12 - loss: 1.5883 - acc: 0.6667

  3/494 [..............................] - ETA: 34:23 - loss: 1.7071 - acc: 0.6889

  4/494 [..............................] - ETA: 32:58 - loss: 1.6005 - acc: 0.7333

...

190/494 [==========>...................] - ETA: 18:25 - loss: 2.0616 - acc: 0.6526

191/494 [==========>...................] - ETA: 18:22 - loss: 2.0615 - acc: 0.6527

192/494 [==========>...................] - ETA: 18:19 - loss: 2.0629 - acc: 0.6521

193/494 [==========>...................] - ETA: 18:16 - loss: 2.0636 - acc: 0.6522

194/494 [==========>...................] - ETA: 18:16 - loss: 2.0630 - acc: 0.6519

In [ ]:

history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)

model.save('small_last4.h5')


Epoch 1/3


  1/494 [..............................] - ETA: 45:07 - loss: 0.9508 - acc: 0.6667

  2/494 [..............................] - ETA: 37:12 - loss: 1.5883 - acc: 0.6667

  3/494 [..............................] - ETA: 34:23 - loss: 1.7071 - acc: 0.6889

  4/494 [..............................] - ETA: 32:58 - loss: 1.6005 - acc: 0.7333

...

190/494 [==========>...................] - ETA: 18:25 - loss: 2.0616 - acc: 0.6526

191/494 [==========>...................] - ETA: 18:22 - loss: 2.0615 - acc: 0.6527

192/494 [==========>...................] - ETA: 18:19 - loss: 2.0629 - acc: 0.6521

193/494 [==========>...................] - ETA: 18:16 - loss: 2.0636 - acc: 0.6522

194/494 [==========>...................] - ETA: 18:16 - loss: 2.0630 - acc: 0.6519

In [23]:


history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)

Epoch 1/3


  1/494 [..............................] - ETA: 45:07 - loss: 0.9508 - acc: 0.6667

  2/494 [..............................] - ETA: 37:12 - loss: 1.5883 - acc: 0.6667

  3/494 [..............................] - ETA: 34:23 - loss: 1.7071 - acc: 0.6889

  4/494 [..............................] - ETA: 32:58 - loss: 1.6005 - acc: 0.7333

...

190/494 [==========>...................] - ETA: 18:25 - loss: 2.0616 - acc: 0.6526

191/494 [==========>...................] - ETA: 18:22 - loss: 2.0615 - acc: 0.6527

192/494 [==========>...................] - ETA: 18:19 - loss: 2.0629 - acc: 0.6521

193/494 [==========>...................] - ETA: 18:16 - loss: 2.0636 - acc: 0.6522

194/494 [==========>...................] - ETA: 18:16 - loss: 2.0630 - acc: 0.6519

In [ ]:

tf = tf.Session(config=tf.ConfigProto(log_device_placement=True))
tf.list_devices()


Epoch 1/3


  1/494 [..............................] - ETA: 45:07 - loss: 0.9508 - acc: 0.6667

  2/494 [..............................] - ETA: 37:12 - loss: 1.5883 - acc: 0.6667

  3/494 [..............................] - ETA: 34:23 - loss: 1.7071 - acc: 0.6889

  4/494 [..............................] - ETA: 32:58 - loss: 1.6005 - acc: 0.7333

...

190/494 [==========>...................] - ETA: 18:25 - loss: 2.0616 - acc: 0.6526

191/494 [==========>...................] - ETA: 18:22 - loss: 2.0615 - acc: 0.6527

192/494 [==========>...................] - ETA: 18:19 - loss: 2.0629 - acc: 0.6521

193/494 [==========>...................] - ETA: 18:16 - loss: 2.0636 - acc: 0.6522

194/494 [==========>...................] - ETA: 18:16 - loss: 2.0630 - acc: 0.6519

In [5]:

model = models.load_model('small_last4.h5')
x_test = []
for f in tqdm(df_test['id'].values):
    img = cv2.imread('data/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))
    
x_test = np.array(x_test, np.float32) / 255.
model.save('small_last.h5')

  0%|          | 0/10357 [00:00<?, ?it/s]

  1%|          | 75/10357 [00:00<00:13, 745.41it/s]

  1%|▏         | 141/10357 [00:00<00:14, 715.62it/s]

  2%|▏         | 215/10357 [00:00<00:14, 722.10it/s]

  3%|▎         | 292/10357 [00:00<00:13, 733.38it/s]

...

 98%|█████████▊| 10119/10357 [00:14<00:00, 702.00it/s]

 98%|█████████▊| 10190/10357 [00:14<00:00, 694.96it/s]

 99%|█████████▉| 10264/10357 [00:14<00:00, 706.38it/s]

100%|█████████▉| 10335/10357 [00:14<00:00, 682.83it/s]

100%|██████████| 10357/10357 [00:14<00:00, 700.75it/s]

In [7]:
preds = model.predict(x_test, verbose=1)

   32/10357 [..............................] - ETA: 15:26

   64/10357 [..............................] - ETA: 13:21

   96/10357 [..............................] - ETA: 12:25

  128/10357 [..............................] - ETA: 11:54

  160/10357 [..............................] - ETA: 11:34

...

10240/10357 [============================>.] - ETA: 7s

10272/10357 [============================>.] - ETA: 5s

10304/10357 [============================>.] - ETA: 3s

10336/10357 [============================>.] - ETA: 1s

10357/10357 [==============================] - 628s 61ms/step


In [9]:
sub = pd.DataFrame(preds)

# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
print('started')
sub.insert(0, 'id', df_test['id'])
sub.to_csv('data/submission_resnet_2.csv', encoding='utf-8', index=False)
print('success')

started


success
